# **전통적인 NLP**



---



- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.
    - **T4 GPU : 16GB VRAM**

- 💡**코드 내용**
    - 언어 AI의 이해를 돕는 코드
    - 2000년대 이전의 자연어 처리(NLP) 과정 이해를 돕는 예제로 구성함
    - 어떤 모델은 실행시간이 오래 걸릴 수 있음.(코랩에서는 RAM을 모두 사용 후 세션이 다운 될 수 있음)


---

## **[1980~90년대]**: 통계적 NLP, 확률론적 접근법

- **핵심 설명**
    - 1980~90년대 자연어처리(NLP)는 규칙 기반 접근법의 한계를 극복하기 위해 **통계적 방법**으로 전환됨
    - **대용량 텍스트 코퍼스의 등장** → **확률 모델 기반 언어 처리** 가능
    - 대표 기법: n-그램 언어 모델, HMM(Hidden Markov Model) 기반 품사 태깅
    - **n-그램 언어 모델** :
        - 앞의 n-1 단어를 이용해 다음 단어의 확률을 추정하는 확률적 언어 모델
    - **HMM(Hidden Markov Model)** :
        - 관찰할 수 있는 데이터(예: 단어)를 통해 그 뒤에 숨겨진 상태(예: 품사)를 추론하는 통계적 모델
        - **베이즈 정리를 시간 순서로 반복 적용하는 모델**
        - 숨겨진 상태(예: 품사) 의 전이 확률과 관측 단어의 발생 확률을 함께 고려하여 텍스트 시퀀스를 모델링하는 확률 기반 모델
        - HMM은 **전이확률(P(tagᵢ|tagᵢ₋₁))** +  **발생확률(P(word|tag))** 을 이용해 가장 가능성 높은 시퀀스를 선택
        - **전이 확률(Transition Probability)**:
            - (마코프 모델이나 HMM에서) 한 상태가 다음 상태로 이동할 확률

### **예제 : n-그램 언어모델**
- Brown Corpus는 1960년대 초반, 미국 브라운 대학교에서 구축됨
- 컴퓨터로 처리 가능하도록 만들어진 **최초의 대규모 영어 텍스트 모음**
- 현대 자연어 처리(NLP) 연구의 초석을 다진 자료
- 총 15개의 장르 (뉴스, 사설, 종교, 기술, 로맨스 소설 등)에서 각각 텍스트 샘플을 추출

In [ ]:
import nltk
from nltk.corpus import brown
from nltk import ngrams
from collections import defaultdict

# 1. NLTK 데이터 다운로드 (최초 1회 실행)
try:
    brown.words()
except LookupError:
    nltk.download('brown')


# 2. 'news' 카테고리 단어 로드 및 소문자 변환
print("Brown 코퍼스 로딩 중...")
words = [word.lower() for word in brown.words(categories='news')]
print(f"총 단어 수: {len(words)}")
print(words)


# 3. 4-gram(Quadgram) 모델 생성
# key: (첫 단어, 두 번째 단어, 세 번째 단어), value: {네 번째 단어: 빈도수}
quadgram_model = defaultdict(lambda: defaultdict(int))


# 단어들을 순회하며 4-gram의 빈도를 계산합니다.
for w1, w2, w3, w4 in ngrams(words, 4, pad_left=True, pad_right=True):
    quadgram_model[(w1, w2, w3)][w4] += 1


# 4. 다음 단어 추론
# "the united states" 다음에 올 단어의 빈도수를 확인합니다.
preceding_words = ("the", "united", "states")
# preceding_words = ("in", "the", "first")
# preceding_words = ("it", "was", "a")
# preceding_words = ("the", "south", "korea")
next_word_candidates = quadgram_model[preceding_words]


# 5. 결과 출력
print("\n" + "="*50)
print(f"✅ 입력 단어: '{preceding_words[0]} {preceding_words[1]} {preceding_words[2]}'")

if next_word_candidates:
    # 빈도수가 높은 순서대로 5개 단어 출력
    most_common_next_words = sorted(next_word_candidates.items(),
                                    key=lambda item: item[1],
                                    reverse=True)
    print("✅ Brown 코퍼스 기반 예측 단어 (빈도수 순):")
    for word, freq in most_common_next_words[:5]:
        print(f"- {word} (빈도: {freq})")
else:
    print(f"Brown 코퍼스에서 '{' '.join(preceding_words)}' 다음에 오는 단어를 찾을 수 없습니다.")
print("="*50)

### **예제 : 대용량 텍스트로 n-그램 생성하여 새로운 문장 생성**
- 이 코드는 **구텐베르크 대용량 텍스트**를 다운로드해서 (--> Project Gutenberg에서 제공하는 고전 문학 전자책 모음)
- bigram 모델(확률적 NLP 기법)을 만들고,
- **조건부 확률 기반으로 새로운 문장을 생성**함

In [ ]:
import re
import random
from collections import Counter, defaultdict
import requests

# 1) 대용량 텍스트 데이터 다운로드 (위키피디아 문서 예시)
url = "https://www.gutenberg.org/files/1342/1342-0.txt"  # 제인 오스틴 'Pride and Prejudice(오만과 편견)'
text = requests.get(url).text.lower()


# 2) 토큰화 (단어 단위)
tokens = re.findall(r"\b\w+\b", text)
#  "단어 경계(\b)로 시작해서, 하나 이상의 단어 문자(\w+)가 나오고, 다시 단어 경계(\b)로 끝나는 문자열을 찾아라."


# 3) n-그램(빅그램) 생성
bigrams = [(tokens[i], tokens[i+1]) for i in range(len(tokens)-1)]
bigram_counts = Counter(bigrams)


# 4) 조건부 확률 계산 (P(w2 | w1))
bigram_model = defaultdict(list)
for (w1, w2), freq in bigram_counts.items():
    bigram_model[w1].append((w2, freq))


# 5) 확률 기반 텍스트 생성
def generate_text(start_word, length=20):
    word = start_word
    result = [word]
    for _ in range(length):
        if word not in bigram_model:
            break
        candidates, weights = zip(*bigram_model[word])
        word = random.choices(candidates, weights=weights, k=1)[0]  # 가중치 값이 큰 것 선택
        result.append(word)
    return " ".join(result)


# 테스트 :
print(f"\n✅ 생성된 텍스트")
result = generate_text("love", 30) # 30개의 (토큰)단어
# result = generate_text("pride", 30)
# result = generate_text("darcy", 30)
print(result)  # 지시어, 단어갯수
len(result.split())


### **예제 : HMM으로 제시어 기반 텍스트 생성**
- **EM(Expectation-Maximization)**
    - HMM의 비지도 학습(unsupervised learning) 방식의 대표적 알고리즘
- **퍼플렉시티(perplexity : PPL)**
    - 언어 모델을 평가하기 위한 평가 지표 -->(헷갈리는 정도를 수치화)
    - 수치가 '낮을수록' 언어 모델의 성능이 좋다.

- $PPL(W)=\sqrt[N]{\frac{1}{\prod_{i=1}^{N}P(w_{i}| w_{i-1})}} $
- **로그우도(Log-Likelihood)**
    - 어떤 모델이 주어진 데이터(여기서는 문장)를 얼마나 잘 설명하는지를 나타내는 지표
    - '우도(Likelihood)'는 모델이 데이터를 생성할 확률을 의미하며,
    - 이 우도 값에 로그(log)를 취한 것이 바로 로그우도
    - 예: 언어 모델이 "I love AI."라는 문장을 생성할 확률
        - 우도(Likelihood): P("IloveAI.")=P("I")xP("love"|"I")xP("AI"|"Ilove")
        - 로그우도(Log-Likelihood): log(P("IloveAI."))=log(P("I"))+log(P("love"|"I"))+log(P("AI"|"Ilove"))
    -  로그 적용 이유
        - 값이 0이 되지 않고 음수로 변환되어 정확한 계산을 유지할 수 있다.
        - 복잡한 곱셈 연산을 단순한 덧셈 연산으로 바꿀 수 있


In [ ]:
# 주의! 실행 시간이 오래 걸린다.
# ============================
# HMM Language Model (unsupervised, EM)
# bigram example:
# ============================
import re, math, random, requests
from collections import Counter, defaultdict
import numpy as np

# ----------------------------
# 0) 데이터 준비 (동일 URL)
# ----------------------------
URL = "https://www.gutenberg.org/files/1342/1342-0.txt"

def fetch_text(url=URL):
    text = requests.get(url).text
    return text

def sentence_tokenize(text):
    # 단어 + 문장부호(.?!)
    toks = re.findall(r"\b\w+\b|[.!?]", text.lower())
    sents, cur = [], []
    for t in toks:
        if t in [".", "!", "?"]:
            if cur:
                sents.append(cur)
                cur = []
        else:
            cur.append(t)
    if cur:  # 마지막 잔여 토큰
        sents.append(cur)
    return sents  # 리스트(문장)들의 리스트(단어)

# ----------------------------
# 1) 어휘 구성 & 희귀어 처리
# ----------------------------
def build_vocab(sentences, min_freq=2, max_tokens=None):
    # sentences: List[List[str]]
    if max_tokens:
        # 대용량일 때 학습 토큰 수 제한(옵션)
        flat = [w for s in sentences for w in s][:max_tokens]
        # 문장도 잘라서 맞춰줌
        words_left = set(flat)
        trimmed = []
        total = 0
        for s in sentences:
            keep = []
            for w in s:
                if total >= max_tokens: break
                if w in words_left:
                    keep.append(w); total += 1
            if keep: trimmed.append(keep)
            if total >= max_tokens: break
        sentences = trimmed

    freq = Counter(w for s in sentences for w in s)
    vocab = {w for w,c in freq.items() if c >= min_freq}
    vocab |= {"<unk>"}  # 희귀어 치환
    word2id = {w:i for i,w in enumerate(sorted(vocab))}
    id2word = {i:w for w,i in word2id.items()}
    # 희귀어 치환
    proc = [[w if w in vocab else "<unk>" for w in s] for s in sentences]
    return proc, word2id, id2word

# ----------------------------
# 2) 시퀀스 변환
# ----------------------------
def to_id_sequences(sentences, word2id):
    return [np.array([word2id[w] for w in s], dtype=np.int32) for s in sentences if len(s) > 0]

# ----------------------------
# 3) 수치 안정화 유틸
# ----------------------------
def logsumexp(v):
    m = np.max(v)
    return m + np.log(np.sum(np.exp(v - m) + 1e-300))

# ----------------------------
# 4) HMM (비지도, EM)
# ----------------------------
class HMM:
    def __init__(self, n_states, vocab_size, seed=42):
        self.K = n_states
        self.V = vocab_size
        rng = np.random.default_rng(seed)

        # 파라미터는 log-확률로 보관
        # 초기분포 pi, 전이 A, 방출 B
        pi = rng.random(self.K); pi /= pi.sum()
        A = rng.random((self.K, self.K)); A /= A.sum(axis=1, keepdims=True)
        B = rng.random((self.K, self.V)); B /= B.sum(axis=1, keepdims=True)

        self.log_pi = np.log(pi + 1e-300)
        self.log_A  = np.log(A + 1e-300)
        self.log_B  = np.log(B + 1e-300)

    # Forward-Backward (한 문장)
    def forward(self, seq):
        T = len(seq)
        alpha = np.full((T, self.K), -np.inf)
        # t=0
        alpha[0] = self.log_pi + self.log_B[:, seq[0]]
        # t>=1
        for t in range(1, T):
            bt = self.log_B[:, seq[t]]
            for k in range(self.K):
                alpha[t, k] = bt[k] + logsumexp(alpha[t-1] + self.log_A[:, k])
        ll = logsumexp(alpha[-1])
        return alpha, ll

    def backward(self, seq):
        T = len(seq)
        beta = np.full((T, self.K), -np.inf)
        beta[-1] = 0.0  # log(1)
        for t in range(T-2, -1, -1):
            bt1 = self.log_B[:, seq[t+1]]
            for k in range(self.K):
                beta[t, k] = logsumexp(self.log_A[k] + bt1 + beta[t+1])
        return beta

    def e_step_accumulate(self, seq, exp_pi, exp_A, exp_B):
        alpha, ll = self.forward(seq)
        beta = self.backward(seq)
        T = len(seq)

        # gamma (state posterior)
        gamma = alpha + beta - ll
        gamma = np.exp(gamma)  # (T,K)

        # xi (pairwise transition posterior) 누적
        for t in range(T-1):
            # (K,K): alpha[t, j] + log_A[j, k]
            s = alpha[t][:, None] + self.log_A
            # (K,K): + log_B[k, w_{t+1}]
            s = s + self.log_B[:, seq[t+1]][None, :]
            # (K,K): + beta[t+1, k]
            s = s + beta[t+1][None, :]
            # normalize in log-space by subtracting ll
            s = s - ll
            xi = np.exp(s)                  # (K,K)
            exp_A += xi                     # 누적

        exp_pi += gamma[0]
        for t in range(T):
            exp_B[:, seq[t]] += gamma[t]

        return ll


    def m_step(self, exp_pi, exp_A, exp_B, alpha_smooth=1e-2):
        # Dirichlet-like 평활화
        exp_pi = exp_pi + alpha_smooth
        exp_A  = exp_A  + alpha_smooth
        exp_B  = exp_B  + alpha_smooth

        self.log_pi = np.log(exp_pi / exp_pi.sum() + 1e-300)

        A = exp_A / exp_A.sum(axis=1, keepdims=True)
        B = exp_B / exp_B.sum(axis=1, keepdims=True)

        self.log_A = np.log(A + 1e-300)
        self.log_B = np.log(B + 1e-300)

    def fit(self, sequences, n_iter=5, alpha_smooth=1e-2, verbose=True):
        for it in range(1, n_iter+1):
            exp_pi = np.zeros(self.K)
            exp_A  = np.zeros((self.K, self.K))
            exp_B  = np.zeros((self.K, self.V))

            total_ll = 0.0
            total_T  = 0
            for seq in sequences:
                ll = self.e_step_accumulate(seq, exp_pi, exp_A, exp_B)
                total_ll += ll
                total_T  += len(seq)

            self.m_step(exp_pi, exp_A, exp_B, alpha_smooth=alpha_smooth)
            avg_nll = - total_ll / max(total_T, 1)
            if verbose:
                print(f"[EM {it}/{n_iter}] avg NLL per token: {avg_nll:.4f} (perplexity ≈ {math.exp(avg_nll):.2f})")

    # 문장 로그우도 (forward의 ll)
    def sequence_loglik(self, seq):
        _, ll = self.forward(seq)
        return ll

    # 무조건 생성
    def sample_unconditional(self, id2word, length=20, seed=0):
        rng = np.random.default_rng(seed)
        # z0 ~ pi, w0 ~ B[z0]
        z = rng.choice(self.K, p=np.exp(self.log_pi))
        words = []
        for t in range(length):
            w = rng.choice(len(id2word), p=np.exp(self.log_B[z]))
            words.append(id2word[w])
            z = rng.choice(self.K, p=np.exp(self.log_A[z]))
        return " ".join(words)

    # 시드 단어를 조건으로 첫 상태를 후험에서 샘플(사후확률)
    def sample_conditioned_on_first_word(self, first_word_id, id2word, length=20, seed=0):
        rng = np.random.default_rng(seed)
        # p(z0 | w0) ∝ pi * B[:, w0]
        post0 = np.exp(self.log_pi + self.log_B[:, first_word_id])
        post0 = post0 / post0.sum()
        z = rng.choice(self.K, p=post0)

        words = [id2word[first_word_id]]
        for _ in range(length-1):
            z = rng.choice(self.K, p=np.exp(self.log_A[z]))
            w = rng.choice(len(id2word), p=np.exp(self.log_B[z]))
            words.append(id2word[w])
        return " ".join(words)

# ----------------------------
# 5) 실행 파이프라인
# ----------------------------
def train_hmm_language_model(
    n_states=8,             # 숨은 상태 수 (주제/문맥 클러스터처럼 작동)
    min_freq=3,            # 희귀어 임계치
    max_tokens=200_000,    # 대용량 시 속도/메모리 제어 (None = 전체 사용)
    em_iters=6,            # EM 반복 횟수
    seed=42
):
    print("\n✅ Downloading text ... (Project Gutenberg #1342)")
    text = fetch_text(URL)
    print("\n✅ Sentence tokenizing ...")
    sentences = sentence_tokenize(text)   # List[List[str]]
    print(f"Total sentences: {len(sentences)}")

    print("\n✅ Building vocab & <unk> ...")
    proc_sents, word2id, id2word = build_vocab(sentences, min_freq=min_freq, max_tokens=max_tokens)
    sequences = to_id_sequences(proc_sents, word2id)
    total_tokens = sum(len(s) for s in sequences)
    print(f"   Vocab size: {len(word2id)}, Tokens used: {total_tokens}")

    hmm = HMM(n_states, vocab_size=len(word2id), seed=seed)
    print("\n✅ Training HMM with EM ...")
    hmm.fit(sequences, n_iter=em_iters, alpha_smooth=1e-2, verbose=True)

    # 상태별 상위 단어(방출 확률 상위)
    B = np.exp(hmm.log_B)
    for k in range(n_states):
        top_ids = np.argsort(-B[k])[:10]
        top_words = [id2word[i] for i in top_ids]
        print(f"\n✅ [State {k}] top words:", ", ".join(top_words))

    return hmm, sequences, word2id, id2word

# ----------------------------
# 6) 데모 실행
# ----------------------------
if __name__ == "__main__":
    hmm, seqs, word2id, id2word = train_hmm_language_model(
        n_states=8, min_freq=3, max_tokens=200_000, em_iters=6, seed=7
    )

    # 무조건 생성
    print("\n=== Unconditional Sample ===")
    print(hmm.sample_unconditional(id2word, length=25, seed=1))


    # 시드 단어 조건 생성 (예: 'love')
    seed_word = "love" if "love" in word2id else "<unk>"
    print("\n=== Conditioned on first word:", seed_word, "===")
    print(hmm.sample_conditioned_on_first_word(word2id[seed_word], id2word, length=25, seed=2))


    # 예시 문장 로그우도(평가)
    # example = ["time","flies","like","an","arrow"]
    # example = [w if w in word2id else "<unk>" for w in example]
    # seq = np.array([word2id[w] for w in example], dtype=np.int32)
    # ll = hmm.sequence_loglik(seq)
    # avg_nll = - ll / len(seq)
    # print(f"\n#Example sentence: {' '.join(example)}")
    # print(f"Log-likelihood: {ll:.3f}, avg NLL/token: {avg_nll:.3f}, perplexity ≈ {math.exp(avg_nll):.2f}")


- **한글일 경우**
    - 데이터 소스: 코드 내부에 한국어 텍스트 포함
    - 토큰화 (Tokenization): 한글은 영어와 달리 단어 경계가 명확하지 않아, 형태소 분석기 사용함( konlpy의 Okt)
    - 코드 구조: 영어 코드와 동일한 train_hmm_language_model 및 HMM 클래스 구조를 유지하여 비교 학습 용이하도록

In [ ]:
!pip install konlpy

In [ ]:
# 주의! 실행 시간이 오래 걸릴 수 있다.
# ============================
# HMM Language Model (unsupervised, EM)
# 한글 bigram example
# ============================
import re, math, random
from collections import Counter, defaultdict
import numpy as np
from konlpy.tag import Okt # konlpy 설치 필요: !pip install konlpy

# ----------------------------
# 0) 데이터 준비 (한글 텍스트)
# ----------------------------
# 실습 편의를 위해 한국어 위키피디아 "한국의 역사" 일부를 사용합니다.
KOREAN_TEXT = """
한국의 역사는 구석기 시대부터 시작된다. 한반도에서는 구석기 시대의 유적들이 여러 곳에서 발견되었다.
이후 신석기 시대에는 농경이 시작되었고, 빗살무늬토기가 사용되었다.
청동기 시대로 접어들면서 고대 사회의 기틀이 마련되었고, 고조선이 건국되었다.
고조선은 한반도 최초의 국가로, 단군왕검에 의해 세워졌다고 전해진다.
고조선 이후 한반도에는 여러 작은 나라들이 등장했다.
이후 삼국 시대가 열리면서 고구려, 백제, 신라가 경쟁하며 발전했다.
고구려는 북방 민족의 침입을 막아내고 넓은 영토를 차지했다.
백제는 해상 무역으로 발전했고, 신라는 삼국을 통일했다.
신라의 삼국 통일은 한민족 역사에 큰 의미를 가진다.
통일 신라 시대는 문화적 발전이 이루어졌으나, 이후 신라는 쇠퇴의 길을 걸었다.
후삼국 시대에는 다시 혼란이 찾아왔고, 고려가 건국되어 후삼국을 통일했다.
고려는 불교를 국교로 삼았고, 다양한 문물을 받아들였다.
몽골의 침입과 간섭을 겪었지만, 고려는 이를 극복하려 노력했다.
이후 조선이 건국되어 새로운 왕조 시대를 열었다.
조선은 유교를 국가의 통치 이념으로 삼았다.
조선 시대에는 과학, 문화, 예술 분야에서 많은 발전이 있었다.
하지만 임진왜란과 병자호란을 겪으며 국력이 쇠퇴했다.
근대에 이르러서는 일본의 식민지배를 받게 되었고,
이후 1945년 해방을 맞이했으나 남북으로 분단되었다.
"""

# konlpy Okt를 사용하여 문장과 단어 토큰화
def sentence_tokenize_korean(text):
    # .!?를 기준으로 문장 분리
    sentences = re.split(r'[.!?]', text)
    sents = []
    okt = Okt()
    for s in sentences:
        # 형태소 분석을 통해 단어들을 리스트로
        tokens = okt.morphs(s.strip())
        if tokens:
            sents.append(tokens)
    return sents

# ----------------------------
# 1) 어휘 구성 & 희귀어 처리 (영어 코드와 동일)
# ----------------------------
def build_vocab(sentences, min_freq=2, max_tokens=None):
    if max_tokens:
        flat = [w for s in sentences for w in s][:max_tokens]
        words_left = set(flat)
        trimmed = []
        total = 0
        for s in sentences:
            keep = []
            for w in s:
                if total >= max_tokens: break
                if w in words_left:
                    keep.append(w); total += 1
            if keep: trimmed.append(keep)
            if total >= max_tokens: break
        sentences = trimmed

    freq = Counter(w for s in sentences for w in s)
    vocab = {w for w,c in freq.items() if c >= min_freq}
    vocab |= {"<unk>"}
    word2id = {w:i for i,w in enumerate(sorted(vocab))}
    id2word = {i:w for w,i in word2id.items()}
    proc = [[w if w in vocab else "<unk>" for w in s] for s in sentences]
    return proc, word2id, id2word

# ----------------------------
# 2) 시퀀스 변환 (영어 코드와 동일)
# ----------------------------
def to_id_sequences(sentences, word2id):
    return [np.array([word2id[w] for w in s], dtype=np.int32) for s in sentences if len(s) > 0]

# ----------------------------
# 3) 수치 안정화 유틸 (영어 코드와 동일)
# ----------------------------
def logsumexp(v):
    m = np.max(v)
    return m + np.log(np.sum(np.exp(v - m) + 1e-300))

# ----------------------------
# 4) HMM (비지도, EM) (영어 코드와 동일)
# ----------------------------
class HMM:
    def __init__(self, n_states, vocab_size, seed=42):
        self.K = n_states
        self.V = vocab_size
        rng = np.random.default_rng(seed)

        pi = rng.random(self.K); pi /= pi.sum()
        A = rng.random((self.K, self.K)); A /= A.sum(axis=1, keepdims=True)
        B = rng.random((self.K, self.V)); B /= B.sum(axis=1, keepdims=True)

        self.log_pi = np.log(pi + 1e-300)
        self.log_A  = np.log(A + 1e-300)
        self.log_B  = np.log(B + 1e-300)

    def forward(self, seq):
        T = len(seq)
        alpha = np.full((T, self.K), -np.inf)
        alpha[0] = self.log_pi + self.log_B[:, seq[0]]
        for t in range(1, T):
            bt = self.log_B[:, seq[t]]
            for k in range(self.K):
                alpha[t, k] = bt[k] + logsumexp(alpha[t-1] + self.log_A[:, k])
        ll = logsumexp(alpha[-1])
        return alpha, ll

    def backward(self, seq):
        T = len(seq)
        beta = np.full((T, self.K), -np.inf)
        beta[-1] = 0.0
        for t in range(T-2, -1, -1):
            bt1 = self.log_B[:, seq[t+1]]
            for k in range(self.K):
                beta[t, k] = logsumexp(self.log_A[k] + bt1 + beta[t+1])
        return beta

    def e_step_accumulate(self, seq, exp_pi, exp_A, exp_B):
        alpha, ll = self.forward(seq)
        beta = self.backward(seq)
        T = len(seq)

        gamma = alpha + beta - ll
        gamma = np.exp(gamma)

        for t in range(T-1):
            s = alpha[t][:, None] + self.log_A
            s = s + self.log_B[:, seq[t+1]][None, :]
            s = s + beta[t+1][None, :]
            s = s - ll
            xi = np.exp(s)
            exp_A += xi

        exp_pi += gamma[0]
        for t in range(T):
            exp_B[:, seq[t]] += gamma[t]

        return ll

    def m_step(self, exp_pi, exp_A, exp_B, alpha_smooth=1e-2):
        exp_pi = exp_pi + alpha_smooth
        exp_A  = exp_A  + alpha_smooth
        exp_B  = exp_B  + alpha_smooth

        self.log_pi = np.log(exp_pi / exp_pi.sum() + 1e-300)

        A = exp_A / exp_A.sum(axis=1, keepdims=True)
        B = exp_B / exp_B.sum(axis=1, keepdims=True)

        self.log_A = np.log(A + 1e-300)
        self.log_B = np.log(B + 1e-300)

    def fit(self, sequences, n_iter=5, alpha_smooth=1e-2, verbose=True):
        for it in range(1, n_iter+1):
            exp_pi = np.zeros(self.K)
            exp_A  = np.zeros((self.K, self.K))
            exp_B  = np.zeros((self.K, self.V))

            total_ll = 0.0
            total_T  = 0
            for seq in sequences:
                ll = self.e_step_accumulate(seq, exp_pi, exp_A, exp_B)
                total_ll += ll
                total_T  += len(seq)

            self.m_step(exp_pi, exp_A, exp_B, alpha_smooth=alpha_smooth)
            avg_nll = - total_ll / max(total_T, 1)
            if verbose:
                print(f"[EM {it}/{n_iter}] avg NLL per token: {avg_nll:.4f} (perplexity ≈ {math.exp(avg_nll):.2f})")

    def sequence_loglik(self, seq):
        _, ll = self.forward(seq)
        return ll

    def sample_unconditional(self, id2word, length=20, seed=0):
        rng = np.random.default_rng(seed)
        z = rng.choice(self.K, p=np.exp(self.log_pi))
        words = []
        for t in range(length):
            w = rng.choice(len(id2word), p=np.exp(self.log_B[z]))
            words.append(id2word[w])
            z = rng.choice(self.K, p=np.exp(self.log_A[z]))
        return " ".join(words)

    def sample_conditioned_on_first_word(self, first_word_id, id2word, length=20, seed=0):
        rng = np.random.default_rng(seed)
        post0 = np.exp(self.log_pi + self.log_B[:, first_word_id])
        post0 = post0 / post0.sum()
        z = rng.choice(self.K, p=post0)

        words = [id2word[first_word_id]]
        for _ in range(length-1):
            z = rng.choice(self.K, p=np.exp(self.log_A[z]))
            w = rng.choice(len(id2word), p=np.exp(self.log_B[z]))
            words.append(id2word[w])
        return " ".join(words)

# ----------------------------
# 5) 실행 파이프라인 (한글 데이터에 맞게 변경)
# ----------------------------
def train_hmm_korean_language_model(
    n_states=8,
    min_freq=2,
    max_tokens=None,
    em_iters=6,
    seed=7
):
    print("Preparing Korean text data...")
    text = KOREAN_TEXT
    print("Sentence tokenizing with Konlpy ...")
    sentences = sentence_tokenize_korean(text)
    print(f"Total sentences: {len(sentences)}")

    print("Building vocab & <unk> ...")
    proc_sents, word2id, id2word = build_vocab(sentences, min_freq=min_freq, max_tokens=max_tokens)
    sequences = to_id_sequences(proc_sents, word2id)
    total_tokens = sum(len(s) for s in sequences)
    print(f"Vocab size: {len(word2id)}, Tokens used: {total_tokens}")

    hmm = HMM(n_states, vocab_size=len(word2id), seed=seed)
    print("Training HMM with EM ...")
    hmm.fit(sequences, n_iter=em_iters, alpha_smooth=1e-2, verbose=True)

    # 상태별 상위 단어
    B = np.exp(hmm.log_B)
    for k in range(n_states):
        top_ids = np.argsort(-B[k])[:10]
        top_words = [id2word[i] for i in top_ids]
        print(f"[State {k}] top words:", ", ".join(top_words))

    return hmm, sequences, word2id, id2word

# ----------------------------
# 6) 데모 실행
# ----------------------------
if __name__ == "__main__":
    hmm, seqs, word2id, id2word = train_hmm_korean_language_model(
        n_states=8, min_freq=2, max_tokens=None, em_iters=6, seed=7
    )

    # 무조건 생성
    print("\n✅=== Unconditional Sample ===")
    print(hmm.sample_unconditional(id2word, length=25, seed=1))

    # 시드 단어 조건 생성 (예: '고려')
    seed_word = "고려" if "고려" in word2id else "<unk>"
    print("\n✅=== Conditioned on first word:", seed_word, "===")
    print(hmm.sample_conditioned_on_first_word(word2id[seed_word], id2word, length=25, seed=2))

### **[참고] 언어 모델 평가 지표**

- 사용 빈도가 높은 순서:
    - Perplexity → BLEU → ROUGE → F1/EM → BERTScore → METEOR → Human Evaluation

| 지표, | 설명, | 주 사용 분야  |
|---|---|---|
| Perplexity (혼란도),| 모델이 텍스트를 얼마나 잘 예측하는지 나타냄. 낮을수록 좋음., | 언어 모델(LM) 확률 기반 평가  |
| BLEU,"| 생성된 문장과 참조 문장의 n-gram 겹침 정도. 0~1, 높을수록 좋음."," | 기계 번역, 문장 생성  | "
| ROUGE,| 참조 요약과 생성 요약 간 단어/n-gram 겹침 비율.," | 텍스트 요약, 생성 모델 | "
| F1-score,| Precision과 Recall의 조화 평균.," | NER, QA, 분류 태스크 | "
| Exact Match (EM),| 예측이 정답과 완전히 일치하는 비율., | 질의응답(QA) |
| BERTScore,| BERT 임베딩으로 생성 텍스트와 참조 간 의미 유사도 측정., | 생성 모델 의미 평가 |
| METEOR,"| BLEU 보완: 동의어, 어간, 정렬 고려."," | 번역, 생성 모델 | "
| Human Evaluation,| 사람이 직접 유창성·일관성·사실성 평가.,|

- **LLM 평가 지표**
    - 언어 모델 자체 품질 → Perplexity
    - 생성 텍스트 품질 → BLEU, ROUGE, BERTScore
    - 추론·지식·상식 → MMLU, HellaSwag, GSM8K, PIQA
    - 대화 품질 → MT-Bench, Chatbot Arena
    - 코딩 성능 → HumanEval, MBPP
    - 안전성 → TruthfulQA, 편향 측정
    - 실무 서비스 관점 → Latency, 비용, 사용자 만족도



---



## **[2000년대]**: 기계학습 기반 NLP


- **핵심 설명**
    - **SVM(Support Vector Machine)의 NLP 적용**
        - 배경: 1995년 Vapnik이 개발한 SVM이 2000년대 초 텍스트 마이닝 분야에서 각광받기 시작했습니다. 고차원 희소 벡터로 표현되는 텍스트 데이터의 특성에 SVM이 매우 적합했기 때문입니다.
        - **핵심 아이디어**:
            - 텍스트를 고차원 벡터 공간으로 매핑
            - 최대 마진을 갖는 결정 경계 찾기
            - 커널 트릭을 통한 비선형 분류
    - **HMM(Hidden Markov Model)의 발전**
        - 배경: 1980년대부터 음성 인식에 사용되던 HMM이 2000년대에 품사 태깅, 개체명 인식 등으로 확장되었습니다.
        - **한계와 극복**:
            - 문제점: 관찰 독립성 가정, 라벨 편향 문제
            - 해결책: CRF(Conditional Random Fields) 개발로 전역 최적화 가능


### **예제 : SVM을 이용한 텍스트 분류**

- SVM(Support Vector Machine) 모델과 **TF-IDF(Term Frequency-Inverse Document Frequency) 벡터화 기법**을 사용하여 텍스트를 분류하는 과정을 보여주는 예제
- 이 코드는 당시의 컴퓨팅 환경과 기술 수준을 반영하여, **텍스트 데이터의 특징을 추출**하고 **분류 모델을 훈련하는 과정**을 보여줌

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

class SVMTextClassifier2000s:
    """2000년대 스타일 SVM 텍스트 분류기: 선형 분류"""

    def __init__(self):
        self.vectorizer = None
        self.classifier = None
        self.pipeline = None

    def create_sample_dataset(self):
        """2000년대 스타일 뉴스 데이터셋 생성"""
        # 실제 2000년대 뉴스 헤드라인 스타일
        news_data = {
            'technology': [
                "Apple introduces new iPod with video capabilities",
                "Google launches Gmail with 1GB storage",
                "Microsoft releases Windows XP Service Pack 2",
                "Yahoo acquires Flickr photo sharing service",
                "Mozilla Firefox browser gains market share",
                "Intel announces dual-core processor technology",
                "Sony PlayStation Portable hits the market",
                "YouTube launches online video platform",
                "Wikipedia becomes popular reference source",
                "Broadband internet adoption increases rapidly"
            ],
            'politics': [
                "President Bush announces new homeland security measures",
                "Election results show divided political landscape",
                "Congress debates healthcare reform legislation",
                "International summit addresses climate change",
                "Supreme Court ruling affects civil rights",
                "Senator proposes new tax reform bill",
                "Presidential candidate announces campaign strategy",
                "Foreign policy experts discuss Middle East crisis",
                "Governor signs education funding legislation",
                "Political analysts predict election outcomes"
            ],
            'sports': [
                "Lakers win NBA championship in overtime thriller",
                "Olympic Games showcase international competition",
                "World Cup final attracts global television audience",
                "Baseball season ends with dramatic playoff series",
                "Tennis tournament features top-ranked players",
                "Football team advances to conference championship",
                "Swimming records broken at international meet",
                "Golf tournament decided on final hole",
                "Basketball coach announces retirement plans",
                "Soccer match ends in penalty shootout"
            ],
            'entertainment': [
                "Hollywood movie breaks box office records",
                "Music industry adapts to digital downloads",
                "Television series finale draws huge audience",
                "Celebrity couple announces engagement news",
                "Film festival showcases independent cinema",
                "Pop star releases highly anticipated album",
                "Broadway show receives critical acclaim",
                "Reality TV show becomes cultural phenomenon",
                "Movie sequel exceeds original's success",
                "Entertainment awards ceremony honors achievements"
            ]
        }

        # 데이터프레임 생성
        texts = []
        labels = []
        for category, articles in news_data.items():
            texts.extend(articles)
            labels.extend([category] * len(articles))

        return pd.DataFrame({'text': texts, 'category': labels})

    def preprocess_text_2000s_style(self, texts):
        """2000년대 스타일 텍스트 전처리"""
        # 당시에는 간단한 전처리만 수행
        processed = []
        for text in texts:
            # 소문자 변환
            text = text.lower()
            # 간단한 정제 (특수문자 제거는 최소화)
            import re
            text = re.sub(r'[^\w\s]', ' ', text)
            text = re.sub(r'\s+', ' ', text).strip()
            processed.append(text)
        return processed

    def train_svm_classifier(self, df):
        """SVM 분류기 훈련 (2000년대 방식)"""
        # 텍스트 전처리
        df['processed_text'] = self.preprocess_text_2000s_style(df['text'])

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            df['processed_text'], df['category'],
            test_size=0.3, random_state=42, stratify=df['category']
        )

        # 2000년대 스타일: TF-IDF + SVM
        self.pipeline = Pipeline([
            ('vectorizer', TfidfVectorizer(
                max_features=1000,
                ngram_range=(1, 2),
                min_df=2,
                max_df=0.8,
                stop_words='english'
            )),
            ('classifier', SVC(
                kernel='linear',
                C=1.0,
                random_state=42,
                # 이 부분을 수정하여 확률 예측을 활성화합니다.
                probability=True
            ))
        ])

        # 모델 훈련
        print("\n=== 2000년대 스타일 SVM 텍스트 분류기 훈련 ===")
        self.pipeline.fit(X_train, y_train)

        # 예측 및 평가
        y_pred = self.pipeline.predict(X_test)

        print("\n✅ 분류 성능 보고서:")
        print(classification_report(y_test, y_pred))

        # 교차 검증 (2000년대 표준 평가 방법)
        cv_scores = cross_val_score(self.pipeline, X_train, y_train, cv=5)
        print(f"\n5-Fold 교차검증 정확도: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

        return X_test, y_test, y_pred

    def analyze_features(self):
        """특성 분석 (2000년대 방식)"""
        if self.pipeline is None:
            print("먼저 모델을 훈련시켜주세요.")
            return

        # 특성 이름과 가중치 추출
        vectorizer = self.pipeline.named_steps['vectorizer']
        classifier = self.pipeline.named_steps['classifier']
        feature_names = vectorizer.get_feature_names_out()

        print("\n✅ === 특성 분석 (SVM 가중치) ===")

        # 각 클래스별 중요 특성 출력
        classes = classifier.classes_
        for i, class_name in enumerate(classes):
            print(f"\n'{class_name}' 클래스의 중요 특성:")

            # 해당 클래스에 대한 가중치
            if hasattr(classifier, 'coef_'):
                # 오직 이 부분만 수정되었습니다.
                # .toarray()를 사용하여 희소 행렬을 밀집 행렬로 변환
                weights = classifier.coef_[i].toarray().flatten() if len(classes) > 2 else classifier.coef_[0].toarray().flatten()

                # 상위 10개 특성
                top_indices = np.argsort(weights)[-10:][::-1]
                for idx in top_indices:
                    print(f"  {feature_names[idx]}: {weights[idx]:.3f}")

# 실행 예제
print("\n\n=== 2000년대 SVM 텍스트 분류 실습 ===\n")

classifier = SVMTextClassifier2000s()

# ----------------------------
# 1. 데이터셋 생성
# ----------------------------
df = classifier.create_sample_dataset()
print(" ✅ 생성된 데이터셋:")
print(df.groupby('category').size())
print("\n✅ 샘플 데이터:")
for category in df['category'].unique():
    print(f"\n{category.upper()}:")
    sample = df[df['category'] == category]['text'].iloc[0]
    print(f"  {sample}")

# ----------------------------
# 2. SVM 모델 훈련 및 평가
# ----------------------------
X_test, y_test, y_pred = classifier.train_svm_classifier(df)

# ----------------------------
# 3. 특성 분석
# ----------------------------
classifier.analyze_features()

# ----------------------------
# 4. 새로운 텍스트 분류 테스트
# ----------------------------
print("\n✅=== 새로운 텍스트 분류 테스트 ===")
test_texts = [
    "Microsoft announces new software development kit",
    "Basketball team wins championship game",
    "Senator proposes new economic policy",
    "Movie star wins academy award"
]

# ----------------------------
# 새로운 텍스트 분류
# ----------------------------
for text in test_texts:
    processed = classifier.preprocess_text_2000s_style([text])

    # 예측 카테고리
    prediction = classifier.pipeline.predict(processed)[0]

    # decision_function을 사용하여 신뢰도 점수 계산
    decision_scores = classifier.pipeline.decision_function(processed)[0]

    # 다중 클래스 분류의 경우, 가장 높은 점수를 신뢰도로 사용
    max_score = np.max(decision_scores)

    print(f"\n➡️ 텍스트: {text}")
    print(f" ➔ 예측 카테고리: {prediction}")
    print(f" ➔ 신뢰도 점수 (decision_function): {max_score:.3f}")

# 신뢰도 점수 (decision_function): 예측된 샘플이 SVM의 결정 경계(decision boundary)로부터 얼마나 멀리 떨어져 있는지를 나타내는 값
#  절대값이 클수록 해당 분류 결과에 대한 모델의 신뢰도가 높다는 의미

### **예제 : SVM을 이용한 스팸메일 분류**

In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

class EmailFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.spam_keywords = [
            'free', 'money', 'win', 'winner', 'cash', 'prize', 'offer',
            'click', 'buy', 'sale', 'discount', 'viagra', 'cialis',
            'mortgage', 'loan', 'credit', 'debt', 'investment',
            'guarantee', 'urgent', 'act now', 'limited time'
        ]

        self.spam_patterns = [
            r'\$+', r'!{2,}', r'\*+', r'#{2,}', r'%+',
            r'[A-Z]{3,}', r'\d+%', r'www\.', r'http://',
            r'\.com', r'\.net', r'\.org'
        ]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """이메일에서 수치 특성 추출"""
        features = []

        for email in X:
            email_features = []
            email_lower = email.lower()

            # 안전한 특성 계산
            try:
                # 1. 스팸 키워드 빈도
                spam_keyword_count = sum(email_lower.count(keyword) for keyword in self.spam_keywords)
                email_features.append(float(spam_keyword_count))

                # 2. 대문자 비율
                if len(email) > 0:
                    upper_ratio = sum(1 for c in email if c.isupper()) / len(email)
                else:
                    upper_ratio = 0.0
                email_features.append(float(upper_ratio))

                # 3. 특수문자 패턴 개수
                special_pattern_count = sum(len(re.findall(pattern, email)) for pattern in self.spam_patterns)
                email_features.append(float(special_pattern_count))

                # 4. 숫자 비율
                if len(email) > 0:
                    digit_ratio = sum(1 for c in email if c.isdigit()) / len(email)
                else:
                    digit_ratio = 0.0
                email_features.append(float(digit_ratio))

                # 5. 평균 단어 길이
                words = email_lower.split()
                if len(words) > 0:
                    avg_word_length = sum(len(word) for word in words) / len(words)
                else:
                    avg_word_length = 0.0
                email_features.append(float(avg_word_length))

                # 6. 느낌표 개수
                exclamation_count = email.count('!')
                email_features.append(float(exclamation_count))

                # 7. URL 개수
                url_count = len(re.findall(r'http[s]?://|www\.', email_lower))
                email_features.append(float(url_count))

                # 8. 이메일 주소 개수
                email_count = len(re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', email))
                email_features.append(float(email_count))

            except Exception as e:
                # 오류 발생 시 기본값으로 설정
                email_features = [0.0] * 8

            features.append(email_features)

        # numpy 배열로 변환 (희소 행렬 문제 방지)
        return np.array(features, dtype=np.float64)

class FixedSVMSpamFilter:
    """SVM 스팸 필터"""

    def __init__(self):
        self.pipeline = None
        self.text_feature_names = None
        self.custom_feature_names = [
            'spam_keywords', 'upper_ratio', 'special_patterns',
            'digit_ratio', 'avg_word_length', 'exclamation_count',
            'url_count', 'email_count'
        ]

    def create_email_dataset(self):
        """이메일 데이터셋 생성"""

        ham_emails = [
            "Hi John, hope you're doing well. Let's meet for coffee this weekend to discuss the project.",
            "The quarterly report is ready for review. Please find the attached document with detailed analysis.",
            "Thank you for your presentation yesterday. The team found it very informative and useful.",
            "Reminder: Staff meeting tomorrow at 10 AM in the conference room. Agenda attached.",
            "Great job on the client presentation! The feedback has been overwhelmingly positive.",
            "Please review the contract terms and let me know if you have any questions or concerns.",
            "The software update is scheduled for this weekend. Expect brief downtime on Sunday morning.",
            "Congratulations on your promotion! Well deserved after all your hard work this year.",
            "Can you send me the latest version of the budget spreadsheet when you have a moment?",
            "The training session was very helpful. Looking forward to implementing these new techniques.",
            "Happy birthday! Hope you have a wonderful day celebrating with family and friends.",
            "The conference next month looks interesting. Would you like to attend together?",
            "Please confirm your attendance for the team dinner on Friday evening at 7 PM.",
            "The new hire orientation is scheduled for Monday. HR will send details soon.",
            "Thanks for covering my shift yesterday. I really appreciate your help and flexibility."
        ]

        spam_emails = [
            "CONGRATULATIONS!!! You've WON $1,000,000!!! Click HERE NOW to claim your PRIZE!!!",
            "URGENT: Your account will be closed! Click www.fakebank.com to verify information NOW!",
            "FREE MONEY! Get $500 cash instantly! No credit check required! Act now!!!",
            "VIAGRA 50% OFF! Best prices guaranteed! Order now and save BIG money!!!",
            "You've been selected for a special offer! Buy now and get 90% discount!!!",
            "MAKE $5000 A WEEK working from home! No experience needed! Start TODAY!!!",
            "WARNING: Your computer is infected! Download our FREE antivirus software NOW!",
            "Lose 20 pounds in 10 days! GUARANTEED results! Order our miracle pills now!",
            "CREDIT PROBLEMS? No problem! Get approved for any loan instantly! Apply now!",
            "FREE iPod! Just pay shipping and handling! Limited time offer - ACT FAST!!!",
            "HOT SINGLES in your area want to meet you! Click here for instant access!",
            "Your mortgage can be cut in HALF! Refinance now and save thousands!!!",
            "FANTASTIC INVESTMENT OPPORTUNITY! Double your money in 30 days! Risk-free!",
            "URGENT: You have unclaimed inheritance of $2.5 million! Contact us immediately!",
            "CASINO BONUS: $200 FREE! No deposit required! Play now and win BIG!!!"
        ]

        emails = ham_emails + spam_emails
        labels = ['ham'] * len(ham_emails) + ['spam'] * len(spam_emails)

        return pd.DataFrame({'email': emails, 'label': labels})

    def preprocess_email(self, emails):
        """이메일 전처리"""
        processed = []
        for email in emails:
            email = re.sub(r'[^\w\s@.-]', ' ', email)
            email = re.sub(r'\s+', ' ', email)
            processed.append(email.strip())
        return processed

    def build_pipeline(self):
        """파이프라인 구성"""

        # 텍스트 특성 파이프라인
        text_pipeline = TfidfVectorizer(
            max_features=500,  # 특성 수 줄임
            ngram_range=(1, 2),
            min_df=1,
            max_df=0.9,
            stop_words='english',
            lowercase=True
        )

        # 전체 파이프라인 (FeatureUnion 사용하지 않음)
        self.pipeline = Pipeline([
            ('tfidf', text_pipeline),
            ('classifier', SVC(
                kernel='linear',
                C=1.0,
                probability=True,
                class_weight='balanced',
                random_state=42
            ))
        ])

        return self.pipeline

    def train_spam_filter(self, df):
        """스팸 필터 훈련"""
        print("\n\n✅ === SVM 스팸 필터 훈련 ===")

        # 데이터 전처리
        df['processed_email'] = self.preprocess_email(df['email'])

        # 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(
            df['processed_email'], df['label'],
            test_size=0.3, random_state=42, stratify=df['label']
        )

        # 파이프라인 구성 및 훈련
        self.build_pipeline()
        self.pipeline.fit(X_train, y_train)

        # 특성 이름 저장
        self.text_feature_names = self.pipeline.named_steps['tfidf'].get_feature_names_out()

        # 성능 평가
        y_pred = self.pipeline.predict(X_test)
        y_prob = self.pipeline.predict_proba(X_test)[:, 1]

        print(f"훈련 데이터: {len(X_train)}개")
        print(f"테스트 데이터: {len(X_test)}개")
        print(f"스팸 비율: {(y_train == 'spam').mean():.2f}")

        print("\n✅ === 성능 평가 ===")
        print(classification_report(y_test, y_pred))

        # ROC AUC
        try:
            roc_auc = roc_auc_score(y_test == 'spam', y_prob)
            print(f"ROC AUC: {roc_auc:.3f}")
        except:
            print("ROC AUC 계산 실패")

        # 교차 검증
        cv_scores = cross_val_score(self.pipeline, X_train, y_train, cv=5, scoring='f1_macro')
        print(f"5-Fold CV F1-Score: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

        return X_test, y_test, y_pred, y_prob

    def analyze_spam_features_fixed(self):
        """스팸 탐지 특성 분석"""
        if self.pipeline is None:
            print("먼저 모델을 훈련시켜주세요.")
            return

        print("\n✅ === 스팸 탐지 중요 특성 분석 ===")

        try:
            # SVM 분류기 가져오기
            classifier = self.pipeline.named_steps['classifier']

            if not hasattr(classifier, 'coef_'):
                print("선형 SVM이 아니므로 가중치를 분석할 수 없습니다.")
                return

            # 가중치 추출 및 numpy 배열로 변환
            coef_matrix = classifier.coef_
            if hasattr(coef_matrix, 'toarray'):
                weights = coef_matrix.toarray()[0]
            else:
                weights = np.array(coef_matrix[0]).flatten()

            feature_names = self.text_feature_names

            print(f"총 특성 수: {len(weights)}")
            print(f"특성 이름 수: {len(feature_names)}")

            # 안전한 인덱스 처리
            max_features = min(len(weights), len(feature_names))

            if max_features == 0:
                print("분석할 특성이 없습니다.")
                return

            # 스팸을 나타내는 상위 특성 (양수 가중치)
            positive_indices = []
            negative_indices = []

            for i in range(max_features):
                weight_val = float(weights[i])  # 스칼라로 변환
                if weight_val > 0:
                    positive_indices.append((i, weight_val))
                elif weight_val < 0:
                    negative_indices.append((i, weight_val))

            # 정렬
            positive_indices.sort(key=lambda x: x[1], reverse=True)
            negative_indices.sort(key=lambda x: x[1])

            print("\n스팸을 강하게 나타내는 특성 (상위 10개):")
            for i, (idx, weight) in enumerate(positive_indices[:10]):
                if idx < len(feature_names):
                    print(f"  {i+1}. {feature_names[idx]}: {weight:.3f}")

            print("\n정상 이메일을 강하게 나타내는 특성 (상위 10개):")
            for i, (idx, weight) in enumerate(negative_indices[:10]):
                if idx < len(feature_names):
                    print(f"  {i+1}. {feature_names[idx]}: {weight:.3f}")

        except Exception as e:
            print(f"특성 분석 중 오류 발생: {e}")
            print("기본 분석을 수행합니다.")

            # 기본 통계 정보
            classifier = self.pipeline.named_steps['classifier']
            print(f"SVM 클래스: {classifier.classes_}")
            print(f"서포트 벡터 수: {classifier.n_support_}")

    def test_new_emails_fixed(self):
        """새로운 이메일 테스트"""
        if self.pipeline is None:
            print("먼저 모델을 훈련시켜주세요.")
            return

        print("\n✅ === 새로운 이메일 스팸 탐지 테스트 ===")

        test_emails = [
            "Hi Sarah, can we reschedule our meeting to Thursday afternoon? Thanks!",
            "FREE CASH NOW!!! Click here to get $1000 instantly! NO QUESTIONS ASKED!!!",
            "The project deadline has been moved to next Friday. Please update your schedules.",
            "URGENT! Your account expires TODAY! Verify now at www.scamsite.com or lose access!",
            "Lunch at the new Italian restaurant was great. Highly recommend their pasta dishes.",
            "VIAGRA CIALIS 80% OFF! Order now and save BIG! Discreet shipping guaranteed!",
            "Please find attached the updated employee handbook. HR policy changes included.",
            "WIN A FREE IPHONE! Just pay $19.95 shipping! Limited time offer - Act NOW!"
        ]

        for i, email in enumerate(test_emails, 1):
            try:
                processed = self.preprocess_email([email])
                prediction = self.pipeline.predict(processed)[0]
                probabilities = self.pipeline.predict_proba(processed)[0]

                # 클래스 순서 확인
                classes = self.pipeline.named_steps['classifier'].classes_
                spam_idx = list(classes).index('spam') if 'spam' in classes else 1
                spam_prob = probabilities[spam_idx]

                print(f"\n{i}. 이메일: {email[:60]}...")
                print(f"   예측: {prediction.upper()}")
                print(f"   스팸 확률: {spam_prob:.3f}")

                confidence = "높음" if max(probabilities) > 0.8 else "보통" if max(probabilities) > 0.6 else "낮음"
                print(f"   신뢰도: {confidence}")

            except Exception as e:
                print(f"\n{i}. 이메일 처리 중 오류: {e}")

    def simple_performance_analysis(self, df):
        """간단한 성능 분석"""
        print("\n✅ === 간단한 성능 분석 ===")

        try:
            # 기본 통계
            print(f"데이터셋 크기: {len(df)}")
            print(f"스팸 비율: {(df['label'] == 'spam').mean():.2f}")

            # 모델 정보
            if self.pipeline:
                classifier = self.pipeline.named_steps['classifier']
                print(f"SVM 커널: {classifier.kernel}")
                print(f"C 파라미터: {classifier.C}")
                print(f"클래스: {classifier.classes_}")

                if hasattr(classifier, 'n_support_'):
                    print(f"서포트 벡터 수: {classifier.n_support_}")

        except Exception as e:
            print(f"분석 중 오류: {e}")

# 실행 예제
print("\n✅=== SVM 스팸 이메일 필터 ===\n")

# 1. 스팸 필터 생성
spam_filter = FixedSVMSpamFilter()

# 2. 데이터셋 생성
email_df = spam_filter.create_email_dataset()
print("이메일 데이터셋:")
print(email_df['label'].value_counts())

print("\n샘플 이메일:")
ham_sample = email_df[email_df['label'] == 'ham']['email'].iloc[0]
spam_sample = email_df[email_df['label'] == 'spam']['email'].iloc[0]
print(f"HAM: {ham_sample[:80]}...")
print(f"SPAM: {spam_sample[:80]}...")

# 3. 스팸 필터 훈련
try:
    X_test, y_test, y_pred, y_prob = spam_filter.train_spam_filter(email_df)
    print("\n모델 훈련 성공!")
except Exception as e:
    print(f"훈련 중 오류: {e}")

# 4. 혼동 행렬
try:
    print("\n✅ === 혼동 행렬 ===")
    cm = confusion_matrix(y_test, y_pred)
    print("혼동 행렬:")
    print(f"        예측")
    print(f"실제    HAM  SPAM")
    print(f"HAM     {cm[0,0]:3d}   {cm[0,1]:3d}")
    print(f"SPAM    {cm[1,0]:3d}   {cm[1,1]:3d}")
except Exception as e:
    print(f"혼동 행렬 생성 오류: {e}")

# 5. 특성 분석
spam_filter.analyze_spam_features_fixed()

# 6. 이메일 테스트
spam_filter.test_new_emails_fixed()

# 7. 간단한 성능 분석
spam_filter.simple_performance_analysis(email_df)

# 8. 2000년대 기술적 특징 요약
print("\n✅ === 2000년대 SVM 스팸 필터 기술적 특징 ===")
print("장점:")
print("  • 고차원 텍스트 데이터에 효과적")
print("  • 마진 최대화로 일반화 성능 우수")
print("  • 확률 기반 신뢰도 제공")
print("  • 비교적 적은 메모리 사용")

print("\n한계점:")
print("  • 대용량 데이터 처리 속도 느림")
print("  • 특성 공학에 과도하게 의존")
print("  • 순차 정보 손실")
print("  • 하이퍼파라미터 튜닝 필요")

print("\n당시 실제 성능:")
print("  • 정확도: 95-98%")
print("  • 오탐률: 1-3%")
print("  • 처리속도: 1000+ 이메일/초")

### **예제 : HMM을 이용한 품사 태깅**
- 1990년대 이전 HMM : 소규모 코퍼스에서 순수 n-gram HMM(빈도추정+간단 unknown 처리)로 Viterbi 태깅이 주류(**가장 가능도 높은 상태(품사)열을 찾아 태그를 부여하는 방법**)
    - 비터비 태깅(Viterbi Tagging): 문장이 주어졌을 때, 각 단어에 가장 적절한 품사(POS) 태그를 찾아내는 동적 계획법(dynamic programming) 기반의 알고리즘
    - 비터비 알고리즘은 HMM에서 관찰된 시퀀스(단어들) 뒤에 숨어있는 가장 가능성 높은 상태 시퀀스(품사 태그들)를 찾는 데 사용됨
    - 즉, 여러 가능한 품사 태그 경로 중 가장 확률이 높은 단 하나의 경로를 효율적으로 찾아냄
- 2000년대 : 대규모 데이터에 형태소·접미사 등 특징+스무딩·반지도·도메인 적응을 얹은 고차 HMM/하이브리드로 **정확도를 끌어올림**

In [ ]:
import nltk
from collections import defaultdict, Counter
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# 필요한 NLTK 데이터 다운로드
try:
    nltk.data.find('corpora/treebank') # 영어 신문 기사 텍스트 코퍼스
except LookupError:
    nltk.download('treebank', quiet=True)

try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger', quiet=True)

class ImprovedHMM2000s:
    """2000년대 개선된 HMM 품사 태거"""

    def __init__(self, smoothing_parameter=0.01):
        self.smoothing = smoothing_parameter
        self.states = set()  # 품사 태그들
        self.observations = set()  # 단어들
        self.initial_prob = defaultdict(float)
        self.transition_prob = defaultdict(lambda: defaultdict(float))
        self.emission_prob = defaultdict(lambda: defaultdict(float))
        self.word_counts = defaultdict(int)
        self.tag_counts = defaultdict(int)
        self.tag_word_counts = defaultdict(lambda: defaultdict(int))

    def create_2000s_training_data(self):
        """2000년대 스타일 훈련 데이터 생성"""
        from nltk.corpus import treebank

        # Treebank 코퍼스 사용 (2000년대 표준)
        tagged_sentences = treebank.tagged_sents()

        # 데이터 전처리 (2000년대 방식)
        processed_sentences = []
        for sentence in tagged_sentences[:1000]:  # 제한된 데이터
            processed_sent = []
            for word, tag in sentence:
                # 간단한 정규화
                word = word.lower()
                # 숫자는 특별 토큰으로 처리
                if word.isdigit():
                    word = '<NUM>'
                # 희소한 단어는 <UNK>로 처리
                elif len(word) == 1 and not word.isalpha():
                    word = '<PUNCT>'

                processed_sent.append((word, tag))

            if processed_sent:  # 빈 문장 제외
                processed_sentences.append(processed_sent)

        return processed_sentences

    def train(self, tagged_sentences):
        """HMM 모델 훈련 (스무딩 적용)"""
        print("\n✅ === 2000년대 개선된 HMM 모델 훈련 ===")

        # 1. 통계 수집
        for sentence in tagged_sentences:
            prev_tag = '<START>'

            for word, tag in sentence:
                # 관찰 및 상태 집합 구축
                self.observations.add(word)
                self.states.add(tag)

                # 카운트 수집
                self.word_counts[word] += 1
                self.tag_counts[tag] += 1
                self.tag_word_counts[tag][word] += 1

                # 초기 확률 (문장 첫 단어)
                if prev_tag == '<START>':
                    self.initial_prob[tag] += 1

                # 전이 확률
                self.transition_prob[prev_tag][tag] += 1

                prev_tag = tag

            # 문장 끝 전이
            self.transition_prob[prev_tag]['<END>'] += 1

        # 2. 확률 정규화 (라플라스 스무딩 적용)
        self._normalize_probabilities()

        print(f"훈련 완료:")
        print(f"  - 상태(태그) 수: {len(self.states)}")
        print(f"  - 관찰(단어) 수: {len(self.observations)}")
        print(f"  - 훈련 문장 수: {len(tagged_sentences)}")

    def _normalize_probabilities(self):
        """확률 정규화 및 스무딩"""
        # 초기 확률 정규화
        total_initial = sum(self.initial_prob.values())
        for tag in self.initial_prob:
            self.initial_prob[tag] /= total_initial

        # 전이 확률 정규화 (라플라스 스무딩)
        for prev_tag in self.transition_prob:
            total_transitions = sum(self.transition_prob[prev_tag].values())
            vocab_size = len(self.states) + 1  # +1 for <END>

            for next_tag in self.transition_prob[prev_tag]:
                count = self.transition_prob[prev_tag][next_tag]
                # 라플라스 스무딩 적용
                self.transition_prob[prev_tag][next_tag] = \
                    (count + self.smoothing) / (total_transitions + self.smoothing * vocab_size)

        # 방출 확률 정규화 (라플라스 스무딩)
        for tag in self.tag_word_counts:
            total_emissions = sum(self.tag_word_counts[tag].values())
            vocab_size = len(self.observations)

            for word in self.tag_word_counts[tag]:
                count = self.tag_word_counts[tag][word]
                # 라플라스 스무딩 적용
                self.emission_prob[tag][word] = \
                    (count + self.smoothing) / (total_emissions + self.smoothing * vocab_size)

    def viterbi_decode(self, sentence):
        """비터비 알고리즘을 이용한 디코딩"""
        words = [word.lower() for word in sentence]
        n = len(words)

        if n == 0:
            return []

        # 미지 단어 처리
        processed_words = []
        for word in words:
            if word not in self.observations:
                if word.isdigit():
                    processed_words.append('<NUM>')
                elif len(word) == 1 and not word.isalpha():
                    processed_words.append('<PUNCT>')
                else:
                    processed_words.append('<UNK>')
            else:
                processed_words.append(word)

        # 동적 계획법 테이블
        dp = defaultdict(lambda: defaultdict(float))
        backpointer = defaultdict(lambda: defaultdict(str))

        # 초기화
        for tag in self.states:
            init_prob = self.initial_prob.get(tag, self.smoothing)
            emission_prob = self._get_emission_prob(tag, processed_words[0])
            dp[0][tag] = np.log(init_prob) + np.log(emission_prob)

        # 순방향 단계
        for t in range(1, n):
            for curr_tag in self.states:
                max_prob = float('-inf')
                best_prev_tag = None

                for prev_tag in self.states:
                    trans_prob = self._get_transition_prob(prev_tag, curr_tag)
                    emission_prob = self._get_emission_prob(curr_tag, processed_words[t])

                    prob = dp[t-1][prev_tag] + np.log(trans_prob) + np.log(emission_prob)

                    if prob > max_prob:
                        max_prob = prob
                        best_prev_tag = prev_tag

                dp[t][curr_tag] = max_prob
                backpointer[t][curr_tag] = best_prev_tag

        # 최적 경로 역추적
        best_path = [''] * n

        # 마지막 태그 찾기
        max_prob = float('-inf')
        for tag in self.states:
            if dp[n-1][tag] > max_prob:
                max_prob = dp[n-1][tag]
                best_path[n-1] = tag

        # 역방향 추적
        for t in range(n-2, -1, -1):
            best_path[t] = backpointer[t+1][best_path[t+1]]

        return list(zip(words, best_path))

    def _get_transition_prob(self, prev_tag, curr_tag):
        """전이 확률 조회 (스무딩 적용)"""
        if prev_tag in self.transition_prob and curr_tag in self.transition_prob[prev_tag]:
            return self.transition_prob[prev_tag][curr_tag]
        else:
            # 미관찰 전이에 대한 스무딩
            total_transitions = sum(self.transition_prob[prev_tag].values()) if prev_tag in self.transition_prob else 0
            vocab_size = len(self.states)
            return self.smoothing / (total_transitions + self.smoothing * vocab_size)

    def _get_emission_prob(self, tag, word):
        """방출 확률 조회 (미지 단어 처리)"""
        if tag in self.emission_prob and word in self.emission_prob[tag]:
            return self.emission_prob[tag][word]
        else:
            # 미관찰 단어에 대한 스무딩
            total_emissions = sum(self.tag_word_counts[tag].values()) if tag in self.tag_word_counts else 1
            vocab_size = len(self.observations)
            return self.smoothing / (total_emissions + self.smoothing * vocab_size)

    def evaluate(self, test_sentences):
        """모델 평가"""
        all_true_tags = []
        all_pred_tags = []

        correct = 0
        total = 0

        for sentence in test_sentences:
            words = [word for word, tag in sentence]
            true_tags = [tag for word, tag in sentence]

            pred_result = self.viterbi_decode(words)
            pred_tags = [tag for word, tag in pred_result]

            all_true_tags.extend(true_tags)
            all_pred_tags.extend(pred_tags)

            # 문장별 정확도
            for true_tag, pred_tag in zip(true_tags, pred_tags):
                if true_tag == pred_tag:
                    correct += 1
                total += 1

        accuracy = correct / total if total > 0 else 0

        print(f"\n✅ === HMM 모델 평가 결과 ===")
        print(f"전체 정확도: {accuracy:.4f}")
        print(f"올바른 예측: {correct}/{total}")

        return accuracy, all_true_tags, all_pred_tags

# 실행 예제
print("✅ === 2000년대 개선된 HMM 품사 태깅 실습 ===\n")

# 1. HMM 모델 생성 및 훈련
hmm_model = ImprovedHMM2000s(smoothing_parameter=0.01)

# 훈련 데이터 준비
training_data = hmm_model.create_2000s_training_data()
print(f"훈련 데이터: {len(training_data)}개 문장")

# 모델 훈련
hmm_model.train(training_data[:800])  # 80% 훈련용

# 2. 테스트 데이터로 평가
test_data = training_data[800:]  # 20% 테스트용
accuracy, true_tags, pred_tags = hmm_model.evaluate(test_data)

# 3. 실제 문장 태깅 테스트
print("\n✅ === 실제 문장 태깅 테스트 ===")
test_sentences = [
    ["The", "quick", "brown", "fox", "jumps"],
    ["Machine", "learning", "is", "very", "interesting"],
    ["I", "am", "studying", "natural", "language", "processing"],
    ["Google", "released", "a", "new", "algorithm"]
]

for sentence in test_sentences:
    result = hmm_model.viterbi_decode(sentence)
    print(f"\n문장: {' '.join(sentence)}")
    print("태깅 결과:")
    for word, tag in result:
        print(f"  {word} → {tag}")

# 4. 성능 비교 (NLTK 기본 태거와 비교)
print("\n✅ === 성능 비교 (NLTK vs 개선된 HMM) ===")

sample_sentence = ["The", "company", "announced", "new", "product", "development"]

# NLTK 기본 태거
try:
    nltk_result = nltk.pos_tag(sample_sentence)
    print(f"NLTK 결과: {nltk_result}")
except:
    print("NLTK 태거 사용 불가")

# 개선된 HMM 결과
hmm_result = hmm_model.viterbi_decode(sample_sentence)
print(f"➡️ 개선된 HMM: {hmm_result}")

### **예제: SVM를 이용한 문장 생성**
- SVM은 생성 모델이 아니라 판별 모델,
- 이전 단어들(컨텍스트) → 다음 단어를 분류하는 방식으로 학습한 뒤,
- 예측을 반복하여 문장을 생성


In [ ]:
# 주의!!! 실행 시간이 오래 걸림 (A!00 16분)
# SVM(LinearSVC)로 "다음 단어 예측"을 반복하여 문장 생성
# 데이터: Project Gutenberg #1342 (Pride and Prejudice)

import re, requests, numpy as np, math, random
from collections import Counter
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from scipy.special import softmax
from scipy.sparse import vstack

# -----------------------------
# 1) 데이터 수집/전처리
# -----------------------------
URL = "https://www.gutenberg.org/files/1342/1342-0.txt"

def fetch_text(url=URL):
    return requests.get(url).text

def sentence_tokenize(text):
    # 단어 + 문장부호(.?!)
    toks = re.findall(r"\b\w+\b|[.!?]", text.lower())
    sents, cur = [], []
    for t in toks:
        if t in [".", "!", "?"]:
            if cur:
                sents.append(cur)
                cur = []
        else:
            cur.append(t)
    if cur:
        sents.append(cur)
    return sents

def build_vocab(sentences, top_k=20000, min_freq=2):
    freq = Counter(w for s in sentences for w in s)
    # 빈도 기준 필터 → 상위 top_k
    candidates = [w for w, c in freq.items() if c >= min_freq]
    candidates.sort(key=lambda w: freq[w], reverse=True)
    vocab = set(candidates[:top_k])
    vocab |= {"<unk>", "<s>"}            # 특수 토큰
    return vocab, freq

def apply_unk(sentences, vocab):
    return [[w if w in vocab else "<unk>" for w in s] for s in sentences]

def add_sentence_markers(sentences):
    # 각 문장 앞에 시작 토큰 <s>를 두 번 붙여 컨텍스트 2개 확보
    return [["<s>", "<s>"] + s for s in sentences if len(s) > 0]

# -----------------------------
# 2) 데이터셋 구성: (w-2, w-1) -> w0
# -----------------------------
def make_dataset(sentences):
    X_feats = []
    y_words = []
    for s in sentences:
        # s: ["<s>","<s>", w0, w1, ...]
        for i in range(2, len(s)):
            w_2, w_1, w0 = s[i-2], s[i-1], s[i]
            # 특징은 dict로 구성 (해싱으로 변환 예정)
            feats = {
                f"w-2={w_2}": 1,
                f"w-1={w_1}": 1,
                f"bigram={w_2}|{w_1}": 1,
            }
            X_feats.append(feats)
            y_words.append(w0)
    return X_feats, y_words

# -----------------------------
# 3) 학습 파이프라인
# -----------------------------
def train_svm_nextword(
    top_k_vocab=20000,
    min_freq=2,
    n_features=2**18,  # FeatureHasher 차원
    test_size=0.05,
    random_state=42
):
    print("Downloading text ...")
    text = fetch_text(URL)

    print("Sentence tokenizing ...")
    sents = sentence_tokenize(text)
    print(f"Total sentences: {len(sents)}")

    print("Building vocab ...")
    vocab, freq = build_vocab(sents, top_k=top_k_vocab, min_freq=min_freq)

    print("Applying <unk> and adding <s> markers ...")
    sents = apply_unk(sents, vocab)
    sents = add_sentence_markers(sents)

    print("Building (context -> next word) dataset ...")
    X_feats, y_words = make_dataset(sents)

    # 해싱 특징 → 희소행렬
    hasher = FeatureHasher(n_features=n_features, input_type="dict")
    X = hasher.transform(X_feats)

    # 레이블 인코딩
    le = LabelEncoder()
    y = le.fit_transform(y_words)

    # 데이터 분할
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    print("Training LinearSVC (multi-class) ...")
    clf = LinearSVC(random_state=random_state, dual="auto")
    clf.fit(X_tr, y_tr)

    acc = clf.score(X_te, y_te)
    print(f"Test accuracy (next-word, top-1): {acc:.4f}")
    return clf, hasher, le, vocab

# -----------------------------
# 4) 문장 생성
# -----------------------------
def context_to_features(w_2, w_1):
    return {f"w-2={w_2}": 1, f"w-1={w_1}": 1, f"bigram={w_2}|{w_1}": 1}

def generate_with_svm(
    clf, hasher, le, vocab,
    seed_word="love",
    max_len=30,
    temperature=0.8,   # 낮출수록 결정적, 높일수록 랜덤
    top_k=10,          # 상위 k 후보에서 샘플
    end_tokens=(".", "!", "?")
):
    # 시작 컨텍스트: <s>, seed
    w_2, w_1 = "<s>", (seed_word if seed_word in vocab else "<unk>")
    result = [w_1]

    for _ in range(max_len):
        feats = context_to_features(w_2, w_1)
        X = hasher.transform([feats])
        scores = clf.decision_function(X)           # shape: (1, n_classes)
        scores = np.asarray(scores).ravel()

        # 상위 k 후보
        top_idx = np.argsort(-scores)[:top_k]
        top_scores = scores[top_idx]

        # SVM 점수 → 확률 비슷하게 샘플링 (softmax with temperature)
        probs = softmax(top_scores / max(temperature, 1e-6))
        next_label = np.random.choice(top_idx, p=probs)
        next_word = le.inverse_transform([next_label])[0]

        result.append(next_word)

        # 종료조건: 문장부호 예측 시 종료
        if next_word in end_tokens:
            break

        # 컨텍스트 이동
        w_2, w_1 = w_1, next_word

    return " ".join(result)

# -----------------------------
# 5) 실행
# -----------------------------
if __name__ == "__main__":
    clf, hasher, le, vocab = train_svm_nextword(
        top_k_vocab=20000, min_freq=2, n_features=2**18, test_size=0.05
    )

    print("\n✅=== SVM으로 문장 생성) ===")

    # 제시어: love
    for i in range(3):
        s = generate_with_svm(
            clf, hasher, le, vocab,
            seed_word="love",
            max_len=30,
            temperature=0.9,
            top_k=8
        )
        print(f"[Gen {i+1}] {s}")





---

